In [2]:
import pandas as pd
import numpy as np
import openpyxl
import pandas_ta as ta
import sys
sys.path.append (r'D:\esharky-corporation\quant team\codes')
import self_code as sc
import Utils as ut
import glob
import warnings
warnings.filterwarnings("ignore")

In [3]:
import yfinance as yf

In [5]:
nifty = yf.download('^NSEI', period='max', interval='1d') # downloading data for nifty through yfinance module
banknifty = yf.download('^NSEBANK', period='max', interval='1d') # downloading data for banknifty through yfinance module

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [6]:
nifty.head(3) # checking dataset for nifty

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-09-17,4518.450195,4549.049805,4482.850098,4494.649902,4494.649902,0
2007-09-18,4494.100098,4551.799805,4481.549805,4546.200195,4546.200195,0
2007-09-19,4550.250000,4739.000000,4550.250000,4732.350098,4732.350098,0


In [7]:
banknifty.head(3) # checking dataset for banknifty

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2007-09-17,6898.000000,6977.200195,6843.000000,6897.100098,6897.020020,0
2007-09-18,6921.149902,7078.950195,6883.600098,7059.649902,7059.567871,0
2007-09-19,7111.000000,7419.350098,7111.000000,7401.850098,7401.764160,0


In [10]:
cdf = pd.DataFrame(columns = ['nf', 'bnf'])
cdf['nf'] = nifty['Close']
cdf['bnf'] = banknifty['Close']
cdf.index = nifty.index

In [11]:
cdf

,nf,bnf
Date,,
2007-09-17,4494.649902,6897.100098
2007-09-18,4546.200195,7059.649902
2007-09-19,4732.350098,7401.850098
2007-09-20,4747.549805,7390.149902
2007-09-21,4837.549805,7464.500000
...,...,...
2023-09-26,19664.699219,44624.199219
2023-09-27,19716.449219,44588.300781
2023-09-28,19523.550781,44300.949219


# cdf is dataframe containing only closing values of both the securities

In [12]:
cdf[['nf','bnf']].corr()

,nf,bnf
nf,1.000000,0.986873
bnf,0.986873,1.000000


In [24]:
cdf['spread'] = np.log(cdf['bnf']) - (np.log(cdf['nf']))
cdf

,nf,bnf,spread,abs spread
Date,,,,
2007-09-17,4494.649902,6897.100098,0.428213,2402.450195
2007-09-18,4546.200195,7059.649902,0.440104,2513.449707
2007-09-19,4732.350098,7401.850098,0.447308,2669.500000
2007-09-20,4747.549805,7390.149902,0.442519,2642.600098
2007-09-21,4837.549805,7464.500000,0.433750,2626.950195
...,...,...,...,...
2023-09-26,19664.699219,44624.199219,0.819451,24959.500000
2023-09-27,19716.449219,44588.300781,0.816018,24871.851562
2023-09-28,19523.550781,44300.949219,0.819385,24777.398438


In [16]:
cdf['abs spread'] = (cdf['bnf']) - (cdf['nf'])

In [25]:
import plotly.express as px
fig = px.line(cdf.spread)
fig.show()

In [17]:
fig = px.line(cdf['abs spread'])
fig.show()

In [29]:
cdf['mean'] = cdf['spread'].rolling(20).mean()
cdf['std'] = cdf['spread'].rolling(20).std()

In [33]:
cdf['z-score'] = (cdf['spread'] - cdf['mean']) / cdf['std']

In [34]:
cdf

,nf,bnf,spread,abs spread,mean,std,z-score
Date,,,,,,,
2007-09-17,4494.649902,6897.100098,0.428213,2402.450195,NaN,NaN,NaN
2007-09-18,4546.200195,7059.649902,0.440104,2513.449707,NaN,NaN,NaN
2007-09-19,4732.350098,7401.850098,0.447308,2669.500000,NaN,NaN,NaN
2007-09-20,4747.549805,7390.149902,0.442519,2642.600098,NaN,NaN,NaN
2007-09-21,4837.549805,7464.500000,0.433750,2626.950195,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-09-26,19664.699219,44624.199219,0.819451,24959.500000,0.823955,0.004195,-1.073532
2023-09-27,19716.449219,44588.300781,0.816018,24871.851562,0.823103,0.003972,-1.783413
2023-09-28,19523.550781,44300.949219,0.819385,24777.398438,0.822727,0.003950,-0.846196


In [35]:
# backtesting code of the hypothesis
pnl = pd.DataFrame(columns = ['Type', 'Entry Price', 'Entry Time', 'Exit Price', 'Exit Time', 'Points', 'Pnl'])
status = 0
entry_price = 0
entry_time = 0

for row, value in cdf.iterrows():
    if status == 0 and value['z-score'] > 2:
        entry_price = value['bnf']
        entry_time = row
        status = 1
        continue
    elif status == 1 and value['z-score'] < 1:
        pnl.loc[len(pnl)] = ['Long bnf', entry_price, entry_time, value['bnf'], row, value['bnf'] - entry_price, (value['bnf'] - entry_price) * 25]
        status = 0
        continue

In [39]:
sc.final(pnl, r'O:\Work\sssion\result.xlsx', capital = 100000)